In [77]:
import pandas

columns = ['face_x', 'face_y','face_width','face_height','face_cam_x', 'face_cam_y','face_cam_z',
          'torso_x','torso_y','torso_width','torso_height','torso_cam_x','torso_cam_y','torso_cam_z',
          'person_x','person_y','person_width','person_height','person_cam_x','person_cam_y','person_cam_z',
        'mean_cam_z']
data = pandas.read_csv('2020-03-12 15:36', header=None)
df = pandas.DataFrame(data.values, columns=columns)

df.head()

,face_x,face_y,face_width,face_height,face_cam_x,face_cam_y,face_cam_z,torso_x,torso_y,torso_width,...,torso_cam_y,torso_cam_z,person_x,person_y,person_width,person_height,person_cam_x,person_cam_y,person_cam_z,mean_cam_z
0,570.0,156.0,59.0,46.0,3.222619,-0.699850,6.235,-1.0,-1.0,-1.0,...,NaN,NaN,494.0,136.0,342.0,144.0,3.890177,-0.371194,6.124,6.1795
1,-1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,-1.0,-1.0,-1.0,...,NaN,NaN,359.0,130.0,348.0,190.0,2.626809,-0.200039,6.592,6.5920
2,294.0,116.0,83.0,54.0,0.035208,-0.143189,0.812,-1.0,-1.0,-1.0,...,NaN,NaN,-1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,0.8120
3,264.0,68.0,111.0,76.0,0.077445,-1.279389,5.282,166.0,143.0,333.0,...,0.319593,8.995,-1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,7.1385
4,519.0,11.0,156.0,116.0,2.151638,-1.291524,4.192,308.0,107.0,371.0,...,0.264356,6.468,-1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,5.3300


In [114]:
df.to_csv('/home/konstantin/Documents/Rofls/data.csv',index=False,header=True)

In [78]:
import numpy as np

# preparing data
face_X = df.iloc[:,[0,1,2,3]]       # face_x,   face_y,   face_width,   face_height
torso_X = df.iloc[:,[7,8,9,10]]     # torso_x,  torso_y,  torso_width,  torso_height
person_X = df.iloc[:,[14,15,16,17]] # person_x, person_y, person_width, person_height

face_y = df.iloc[:,6]
torso_y = df.iloc[:,13]
person_y = df.iloc[:,20]

In [79]:
def dropNa(X,y, inplace=True):
    drop_list = []

    for i in range(len(y)):
        if np.isnan(y[i]):
            drop_list.append(i)

    _X = X.drop(drop_list,inplace=inplace)
    _y = y.dropna(inplace=inplace)
    
    if inplace:
        return (None,None)
    else:
        return (_X,_y)

In [80]:
for X,y in ((face_X,face_y), (torso_X,torso_y), (person_X,person_y)):
    dropNa(X,y)

In [97]:
from sklearn.model_selection import cross_val_score

# Best possible score is 1.0 and it can be negative 
# (because the model can be arbitrarily worse). 
# A constant model that always predicts the expected value of y, 
# disregarding the input features, would get a R^2 score of 0.0.

def get_scores(X, y, cv, estimator, scoring='r2'):

    return cross_val_score(estimator=estimator, 
                        X=X, 
                        y=y, 
                        cv=cv,
                        scoring=scoring)

In [98]:
#  random forest Regressor
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

kf = KFold(n_splits=5, shuffle=True, random_state=1)

face_scores_rfr   = get_scores(face_X,   face_y,   kf, RandomForestRegressor(n_estimators=30,random_state=1) )
torso_scores_rfr  = get_scores(torso_X,  torso_y,  kf, RandomForestRegressor(n_estimators=30,random_state=1) )
person_scores_rfr = get_scores(person_X, person_y, kf, RandomForestRegressor(n_estimators=30,random_state=1) )

print('Face: ',   face_scores_rfr)
print('Torso: ',  torso_scores_rfr)
print('Person: ', person_scores_rfr)

('Face: ', array([ 0.24235095, -0.05853207,  0.21562556,  0.11668463,  0.01242137]))
('Torso: ', array([ 0.04732474,  0.18592774,  0.10949628, -0.22558165,  0.20154389]))
('Person: ', array([ 0.32956805,  0.00571563,  0.05568167, -0.27921136,  0.01079175]))


In [100]:
# linear regression
from sklearn.linear_model import LinearRegression

face_scores_lr   = get_scores(face_X,   face_y,   kf, LinearRegression(normalize=True) )
torso_scores_lr  = get_scores(torso_X,  torso_y,  kf, LinearRegression(normalize=True) )
person_scores_lr = get_scores(person_X, person_y, kf, LinearRegression(normalize=True) )

print('Face: ',   face_scores_lr)
print('Torso: ',  torso_scores_lr)
print('Person: ', person_scores_lr)

('Face: ', array([ 0.02357517, -0.0020204 , -0.02496094, -0.05339538, -0.03689321]))
('Torso: ', array([ 0.10625176,  0.05783975,  0.10063989, -0.0951957 ,  0.07632126]))
('Person: ', array([ 0.12525503,  0.0959522 ,  0.04281347, -0.59495248, -0.0291822 ]))
